In [1]:
import requests
import pandas as pd
import json
from datetime import datetime

In [2]:
url = "https://beanyfood.mysapogo.com/admin/stock_transfers.json?page=1&limit=20"
cookie = "report_ver=3; _fw_crm_v=2861336f-94de-4be5-ec8a-7fa035b27646; _fbp=fb.1.1722054266002.361545149807384444; _hjSessionUser_1081489=eyJpZCI6IjY5MTBjMDYxLTZiMTYtNTNlYy1iZDNhLTEwYjhjMDUxMzFkMCIsImNyZWF0ZWQiOjE3MjIwNzQwODA1OTksImV4aXN0aW5nIjp0cnVlfQ==; _gid=GA1.2.1281807505.1727183385; _gat_gtag_UA_66880228_3=1; sapo-admin=CfDJ8DA1016STPdNrAaTdK2HgWc5ryW-K_oBZ6EN2A24GF_Hl8jwZ1TqUCd1BKIAbbkF4mJtFGm5Ygap4Bd0g5TY0SmmO_kzxpg1dRulsc5CU1rPChtsLs2QBcPaUdmsAUo7N7DH1hGmOgfd0g_TpvwfJ-PVUvRep8MFZvTx2Gtj8d35fazhHGw2AuK4vpM8GPFBY9lyUY_485AunJ6PFK0yQnsw5N6sdCiPLgn1qmvbYGP9hvqTKBzhsdbSimmv8hw5UYywRJdBhUk_Dkj0Gwiai6juvQ6YL479BY0Q0atLVOnylrCD_Jat_y0A5B5CJV4LOOgMD8jslY5R6-GZmqRREarsWhbQ31s-D9N8Ec5GBgTJEqdaKwxFGyTvMSinLK4PbNcVemx0QHRM75t7twAPZz3yDQ2awhPgc2N7im_D4-kbriVPxlTWnVZhv1BJc9ePNAPYB1s-CYkVnlzSwl2HieGLxnflg6_yHHcyCX1li2qcO2huq7kPFTmTW3Xa5fMe7pqyHOlzIhfcKxxUA73LCA_VMUPXfn66dBHRULtfzZbYs97kWdvPPoiVcrTmqv4M7h9xRb1ShPdG6kKJ9T5IKZ3nheAGRde69i5PPtPTKLaEbqqjNmR_ilvY80uQ1hE7PMaLPdmCTsWX05F4vl42z0iuUtCDnmHB7l61DHJQmb56MNUpV2mpdHoEFBbHaakRaPB0fkq7LshlUtK6pONeIcCW9GwtQCXo-IdKv2w_mFCLOyEYFMuSJgRzX47Zw2-Ijg; lang=vi; _admin_session_id=df2d968a-adfb-7d59-9268-dfa2e14d5b7a; .AspNetCore.Session=CfDJ8DA1016STPdNrAaTdK2HgWcyXOnZjncFf8zRNY3yIFenTzg7IuxvEf8T2s9e4%2BBT71fqjOJLJ%2FKEm3yraZlBbcXJAnsl3WNfs36Mv8xxgz9Mn590ff3tPL38nvXXd7mvoiUkNFRyIrxF9%2FkWPIsiq7xRjWyASA1q61OaSR9tC9Aj; _ga_4NX0F91DEX=GS1.1.1727183385.57.1.1727183386.0.0.0; _ga=GA1.1.415576317.1707995805; _ga_Z1V9QDBF2Q=GS1.1.1727183385.60.1.1727183386.0.0.0"
payload = {}
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'vi',
  'Connection': 'keep-alive',
  'Cookie': cookie,
  'Referer': 'https://beanyfood.mysapogo.com/admin/stock_transfers.json?page=1&limit=20',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
  'X-Sapo-Client': 'sapo-frontend-v3',
  'X-Sapo-ServiceId': 'sapo-frontend-v3',
  'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"'
}

response = requests.request("GET", url, headers=headers, data=payload)

In [ ]:
data = json.loads(response.text)

In [ ]:
for stock_transfers in data['stock_transfers']:
    created_on = stock_transfers['created_on']
    code = stock_transfers['code']
    for line_item in stock_transfers['line_items']:
        lots_dates = line_item["lots_dates"]
        for lots_date in lots_dates:
            lots_number = lots_date["lots_number"]
            manufacture_date = lots_date["manufacture_date"]
            expiration_date = lots_date["expiration_date"]
            type_st = "CHUYỂN"
            quantity = lots_date["quantity"]
            